In [1]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# 回测框架
import backtrader as bt

### 基础函数

In [2]:
#s_date = (datetime.datetime.now() - datetime.timedelta(days = 365))
#e_date = datetime.datetime.now()

def get_data_index(token, stock_code, s_date, e_date):
    ts.set_token(token)
    pro = ts.pro_api()
    df = pro.index_daily(ts_code=stock_code, 
                       fields='trade_date,open,high,low,close,vol',
                       start_date=s_date.strftime('%Y%m%d'), end_date=e_date.strftime('%Y%m%d'))
    df.index=pd.to_datetime(df.trade_date)
    df.drop('trade_date', axis=1, inplace = True)
    df = df.astype('float')
    df.rename(columns = {'vol':'volume'}, inplace = True)
    df['openinterest'] = 0
    return df.iloc[::-1]


def get_data_ts(token, stock_code, s_date, e_date):
    ts.set_token(token)
    pro = ts.pro_api()
    df = pro.daily(ts_code=stock_code, 
                       fields='trade_date,open,high,low,close,vol',
                       start_date=s_date.strftime('%Y%m%d'), end_date=e_date.strftime('%Y%m%d'))
    df.index=pd.to_datetime(df.trade_date)
    df.drop('trade_date', axis=1, inplace = True)
    df = df.astype('float')
    df.rename(columns = {'vol':'volume'}, inplace = True)
    df['openinterest'] = 0
    return df.iloc[::-1]

### MACD策略一号

In [3]:
class Strategy_MACD1(bt.Strategy):
    
    params = (
        ('printlog', False),
        ('ema_short', 12),
        ('ema_long', 26),
        ('macd_period', 9),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            with open('log.txt', 'a') as file:
                file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):
        
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # Add MACD indicator
        self.macdhisto =  bt.indicators.MACDHisto(self.datas[0])
        #self.macdhisto =  bt.indicators.MACDHisto(self.params.ema_short, self.params.ema_long, self.params.macd_period)
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                
                self.log('BUY EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                         (order.executed.price,
                          order.executed.size,
                          self.broker.get_cash(),
                          self.broker.get_value()))
            else:  # Sell

                self.log('SELL EXECUTED, Price: %.2f, Lot:%i, Cash: %i, Value: %i' %
                        (order.executed.price,
                          -order.executed.size,
                          self.broker.get_cash(),
                          self.broker.get_value()))

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        #self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
        
    def next(self):
        
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        #if self.order:
        #    return
           
        # MACD Buy Signal
        if self.macdhisto.histo[0] > 0 and self.macdhisto.histo[-1] < 0:
            self.log('BUY CREATE, Price: %.2f, Lots: %i, Current Position: %i' % (self.dataclose[0], 
                                                                                         100, self.getposition(self.data).size))
            self.buy(size = 100)
                   
        # MACD Sell Singal
        elif self.macdhisto.histo[0] < 0 and self.macdhisto.histo[-1] > 0:
                if self.getposition(self.data).size > 0:
                    self.log('SELL CREATE (Close), Price: %.2f, Lots: %i' % (self.dataclose[0], 
                                                                                            self.getposition(self.data).size))
                    self.close()
                    
        # Keep track of the created order to avoid a 2nd order
        #self.order = self.sell(size = self.getposition(data).size - opt_position)                   

### 回测

In [4]:
if __name__ ==  '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    strats = cerebro.addstrategy(Strategy_MACD1, printlog = True) 

    s_date = datetime.datetime.now() - datetime.timedelta(days = 365)
    e_date = datetime.datetime.now()

    # Create stock Data Feed
    stock_index = '002057.SZ'
    df = get_data_ts('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6', 
                        stock_index, s_date, e_date)
    data = bt.feeds.PandasData(dataname=df,fromdate=s_date,todate=e_date)

    # Add the index Data Feed to Cerebo
    cerebro.adddata(data)

    # Single stock testing 
#    df = get_data_ts('74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6', 
#                        '000001.SZ', s_date, e_date)
#    data = bt.feeds.PandasData(dataname=df,fromdate=s_date,todate=e_date)
#    cerebro.adddata(data, name = '600600.SH')
    
    # Set cash inside the strategy
    cerebro.broker = bt.brokers.BackBroker(coc=True)   
    cerebro.broker.setcash(10000)

    # Set leverage
    #cerebro.broker.setcommission()

    # Print out the starting conditions
    start_value = cerebro.broker.getvalue()
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
2021-12-16: BUY CREATE, Price: 9.41, Lots: 100, Current Position: 0
2021-12-17: BUY EXECUTED, Price: 9.41, Lot:100, Cash: 9059, Value: 9981
2021-12-29: SELL CREATE (Close), Price: 9.12, Lots: 100
2021-12-30: SELL EXECUTED, Price: 9.12, Lot:100, Cash: 9971, Value: 9971
2021-12-30: OPERATION PROFIT, GROSS -29.00, NET -29.00
2022-02-14: BUY CREATE, Price: 8.29, Lots: 100, Current Position: 0
2022-02-15: BUY EXECUTED, Price: 8.29, Lot:100, Cash: 9142, Value: 9975
2022-03-07: SELL CREATE (Close), Price: 8.60, Lots: 100
2022-03-08: SELL EXECUTED, Price: 8.60, Lot:100, Cash: 10002, Value: 10002
2022-03-08: OPERATION PROFIT, GROSS 31.00, NET 31.00
2022-03-24: BUY CREATE, Price: 8.53, Lots: 100, Current Position: 0
2022-03-25: BUY EXECUTED, Price: 8.53, Lot:100, Cash: 9149, Value: 10026
2022-04-08: SELL CREATE (Close), Price: 8.34, Lots: 100
2022-04-11: SELL EXECUTED, Price: 8.34, Lot:100, Cash: 9983, Value: 9983
2022-04-11: OPERATION PROFIT, GROSS -19.00, NET